# PowerShell Download from URL
Detects a Powershell process that contains download commands in its command line string

## Rule Content
```
- title: PowerShell Download from URL
  id: 3b6ab547-8ec2-4991-b9d2-2b06702a48d7
  status: experimental
  description: Detects a Powershell process that contains download commands in its
    command line string
  author: Florian Roth
  tags:
  - attack.t1086
  - attack.execution
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      Image: '*\powershell.exe'
      CommandLine:
      - '*new-object system.net.webclient).downloadstring(*'
      - '*new-object system.net.webclient).downloadfile(*'
      - '*new-object net.webclient).downloadstring(*'
      - '*new-object net.webclient).downloadfile(*'
    condition: selection
  fields:
  - CommandLine
  - ParentCommandLine
  falsepositives:
  - unknown
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_path.keyword:*\\powershell.exe AND process_command_line.keyword:(*new\-object\ system.net.webclient\).downloadstring\(* OR *new\-object\ system.net.webclient\).downloadfile\(* OR *new\-object\ net.webclient\).downloadstring\(* OR *new\-object\ net.webclient\).downloadfile\(*))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()